In [2]:
import spacy

In [3]:
import pandas as pd

In [4]:
modelo_spacy = spacy.load("pt_core_news_sm")

lista_cargos = ["Gerente de Projetos", "Analista de Dados", "Diretor", "gestora", "engenheiro de machine learning"]
lista_termos_tec = ["banco de dados", "Inteligência Artificial", "Machine Learning", "BigData", "llms", "sistema"]
lista_areas = ["saúde", "financeiro", "TI", "recursos humanos"] 


textos_entrada = [
    {
        "id": 1,
        "mensagem": "Na reunião de 08 de julho, a Dra. Ana Costa, Gerente de Projetos, discutiu a implementação de Inteligência Artificial no setor de saúde da empresa TechSolutions Ltda."
    },
    {
        "id": 2,
        "mensagem": "O Sr. Pedro Rocha, Analista de Dados, apresentou o relatório de desempenho do sistema BigData, com previsão de atualização para 20 de agosto, na filial de São Paulo."
    }
]



In [5]:
# Rotular entidades costumizadas com o phrase matcher
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(modelo_spacy.vocab)

matcher.add("CARGO", [modelo_spacy.make_doc(text) for text in lista_cargos])
matcher.add("TERMO_TEC", [modelo_spacy.make_doc(text) for text in lista_termos_tec])
matcher.add("AREA", [modelo_spacy.make_doc(text) for text in lista_areas])


In [6]:
lista_entidades_coletadas = [] 

for item_texto in textos_entrada:
    id_atual = item_texto["id"]
    mensagem_atual = item_texto["mensagem"]
    
    doc = modelo_spacy(mensagem_atual) 
    
    
    for ent in doc.ents:
        lista_entidades_coletadas.append({
            "id": id_atual,
            "mensagem": mensagem_atual,
            "tipoDaEntidade": ent.label_, 
            "entidade": ent.text,
            "posicao_inicio": ent.start_char,
            "posicao_fim": ent.end_char,
            "origem": "SpaCy_Padrao"
        })

print("Entidades padrão do SpaCy coletadas.")

Entidades padrão do SpaCy coletadas.


In [ ]:
# Coletar entidades customizadas via regras do PhraseMatcher

matches = matcher(doc)


for match_id, start_token, end_token in matches:
        span = doc[start_token:end_token] 
        label = modelo_spacy.vocab.strings[match_id] # Converte o ID numérico para o LABEL string (ex: "CARGO")
        print(label)

        lista_entidades_coletadas.append({
            "id": id_atual,
            "mensagem": mensagem_atual,
            "tipoDaEntidade": label, 
            "entidade": span.text,
            "posicao_inicio": span.start_char,
            "posicao_fim": span.end_char,
            "origem": "Regra_Customizada" 
        })
    


CARGO
TERMO_TEC
TERMO_TEC


In [ ]:

df_entidades = pd.DataFrame(lista_entidades_coletadas)


df_organizado = df_entidades.sort_values(by=['id', 'tipoDaEntidade', 'posicao_inicio']).reset_index(drop=True)

print("\nDataFrame final criado e organizado.")



DataFrame final criado e organizado.


In [ ]:
# --- Ajustar opções de exibição para o Jupyter ---
pd.set_option('display.max_colwidth', None) # Não trunca o texto nas colunas
pd.set_option('display.max_rows', None)    # Mostra todas as linhas
pd.set_option('display.width', 1000)      # Define uma largura de tela maior

print("\n--- Visualização do DataFrame de Entidades (Padrão SpaCy e Customizadas Simuladas) ---")

print(df_organizado[['id', 'entidade', 'tipoDaEntidade', 'origem', 'mensagem']])



--- Visualização do DataFrame de Entidades (Padrão SpaCy e Customizadas Simuladas) ---
    id                 entidade tipoDaEntidade             origem                                                                                                                                                                mensagem
0    1      Gerente de Projetos           MISC       SpaCy_Padrao  Na reunião de 08 de julho, a Dra. Ana Costa, Gerente de Projetos, discutiu a implementação de Inteligência Artificial no setor de saúde da empresa TechSolutions Ltda.
1    1  Inteligência Artificial           MISC       SpaCy_Padrao  Na reunião de 08 de julho, a Dra. Ana Costa, Gerente de Projetos, discutiu a implementação de Inteligência Artificial no setor de saúde da empresa TechSolutions Ltda.
2    1                      Dra            ORG       SpaCy_Padrao  Na reunião de 08 de julho, a Dra. Ana Costa, Gerente de Projetos, discutiu a implementação de Inteligência Artificial no setor de saúde da empr